In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
DATE_FORMAT = '%m.%Y'

In [3]:
def safe_mkdf(path, cols):
    dateparse = lambda x: pd.datetime.strptime(x, '%m.%Y')
    if os.path.exists(path):
        df = pd.read_csv(path, index_col=0, parse_dates = [1], date_parser=dateparse)
    else:
        df = pd.DataFrame(columns=cols)
    return df

In [4]:
class Hist_df():
    hist_columns = ['Date', 'Amount']
    
    
    def __init__(self, path):
        self.path = path
        self.df = safe_mkdf(path, Hist_df.hist_columns)
        
    def add_row(self, string_date, amnt):
        date_tm = pd.to_datetime(string_date, format=DATE_FORMAT)
        index = self.df.index.size
        self.df.loc[index] = pd.Series({Hist_df.hist_columns[0]: date_tm, Hist_df.hist_columns[1]: amnt})
        self.df.to_csv(self.path, date_format= DATE_FORMAT)
        
    def sort(self):
        self.df = self.df.sort_values(by = 'Date')
        self.df.index = np.arange(self.df.index.size)
        

In [19]:
def check_dates(hist_df):
    return not hist_df.df['Date'].duplicated().any()

In [6]:
class Investment():
    
    def safe_mkdir(self, path):
        if not os.path.exists(path):
            os.makedirs(path)
            
    def __init__(self, path):
        self.safe_mkdir(os.path.expanduser(path))
        
        self._stat_hist_filename = os.path.join(os.path.expanduser(path),'stat_hist.csv')
        
        self._deps_hist_filename = os.path.join(os.path.expanduser(path),'deps_hist.csv')
        
        self._withs_hist_filename = os.path.join(os.path.expanduser(path),'withs_hist.csv')
        
        self.stat_hist_df = Hist_df(self._stat_hist_filename)
        self.deps_hist_df = Hist_df(self._deps_hist_filename)
        self.withs_hist_df = Hist_df(self._withs_hist_filename)
        
    def add_deposit(self, string_date, amnt):
        self.deps_hist_df.add_row(string_date, amnt)
        self.deps_hist_df.sort()
        
    def add_state(self, string_date, amnt):
        self.stat_hist_df.add_row(string_date, amnt)
        self.stat_hist_df.sort()        
        if not check_dates(self.stat_hist_df):
            print("Warining! There is duplicates in state history")

In [28]:
invest = Investment('./data')
#invest.add_deposit('04.2017', 1000)
invest.add_state('05.2017', 15000)

In [29]:
print(invest.stat_hist_df.df)

        Date  Amount
0 2017-01-01   15000
1 2017-02-01   15000
2 2017-05-01   15000


In [38]:
r=pd.DataFrame(columns=['x', 'y'])
r.loc[0] = [1,2]
r

,x,y
0,1,2
